# SNS 데이터 분석 - 그룹 나누기

## 문장 분류

---

In [ ]:
import os
import numpy as np
import pandas as pd

from gensim.models import Word2Vec


### 사용자 설정

In [ ]:
# word2vec 모델 파일
fname_model   = 'data/w2v_model.bin'

# 그룹별 키워드 정의 파일
fname_group   = 'data/group_101.xlsx'

# 형태소 분류된 컨텐츠 파일
#fname_content = 'data/morph_101.txt'
fname_content = 'data/morph_101.txt'

# 그룹별 분류 결과 파일
fname_result  = 'data/result_101.csv'

# 문장전체에 대해 매칭된 단어의 수가 WSCORE_MAX 이상이어야 그 그룹으로 채택 
WSCORE_MAX = 5

# 유사도 비교시 사용할 기준값
SIM_THRESHOLD = 0.6

### 단어 비교 함수 정의

In [ ]:
def cmp_word( word1, word2, sim_std = 0.7):
    try:
        sim = w2v_model.similarity(word1, word2)
        if sim >= sim_std:
            return True
        else:
            return False
    except:
        return False

### Word2Vec  모델 로드

In [ ]:
# Load Word2Vec model
w2v_model = Word2Vec.load(fname_model)


### 그룹별 키워드 정의 파일 로드

In [ ]:
df_g = pd.read_excel(fname_group, encoding='cp949')

# Initialize
df_g['wscore'] = 0
df_g

### 컨텐츠 파일 로드

In [ ]:
df_c = pd.read_csv(fname_content, sep='\t', header=None, encoding='utf-8')

df_c.rename(columns= {0:'doc_id', 1:'content'}, inplace=True)
df_c.set_index('doc_id', inplace=True)

# Initialize
df_c['gid'] = 0
df_c['gname'] = '알수없는그룹'
df_c

### 스코어 설정 함수 정의

In [ ]:
def set_score( word ):
    for idx, key_lst in enumerate(df_g['keyword']):
        for keyword in key_lst.split(','):
            if cmp_word(word, keyword, sim_std = SIM_THRESHOLD):
                df_g['wscore'].iloc[idx] += 1
                break
                

### 컨텐츠 파일 내용 그룹별 분류

In [ ]:
%%time
for i, line in enumerate(df_c['content']):
    df_g['wscore'] = 0
    if not isinstance(line,str): continue
    for word in line.split():
        set_score(word)
        
    wscore_max = df_g['wscore'].max()
    if wscore_max >= WSCORE_MAX:
        df_t = df_g[df_g['wscore'] == wscore_max]
        df_c['gid'].iloc[i] = df_t['gid'].iloc[0]
        df_c['gname'].iloc[i] = df_t['gname'].iloc[0]
    #
    # FDNX FIXME: For testing...
    #
    if i == 10 : break
    

In [ ]:
df_c

### 그룹별 분류 결과 파일 저장

In [ ]:
df_c.to_csv(fname_result, encoding='cp949')

### <참고> Word2Vec 유사도

In [ ]:
w2v_model.wv.similarity('어미', '동물')

In [ ]:
w2v_model.wv.most_similar('강아지', topn=10)

In [ ]:
# end of file